In [1]:
# jax.config.update('jax_platform_name', 'cpu')
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_disable_jit", True)
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
%matplotlib widget
log_folder = 'runs'

#from matplotlib import rc
### for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
#rc('text', usetex=True)
#rc('axes', titlesize=16)
#rc('axes', labelsize=16)
#rc('axes', axisbelow=False)
#rc('lines',linewidth=1.5)
# lines.markersize : 10
#rc('xtick', labelsize=16)
#rc('xtick.major',size=10)
#rc('xtick.minor',size=5)
#rc('xtick',direction='in')
#rc('ytick', labelsize=16)
#rc('ytick.major',size=10)
#rc('ytick.minor',size=5)
#rc('ytick',direction='in')
# rc('legend',fontsize='large')
import jax
jax.config.update("jax_enable_x64", True)


/home/ohahn/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
from pylinger_background import evolve_background
from pylinger_perturbations import evolve_perturbations, get_xi_from_P, power_Kaiser, get_power

In [3]:
## Cosmological Parameters

## Cosmological Parameters
Tcmb    = 2.7255
YHe     = 0.248
Omegam  = 0.32
Omegab  = 0.05
OmegaDE = 1.0-Omegam
w_DE_0  = -0.9999
w_DE_a  = 0.00
cs2_DE  = 0.9999
num_massive_neutrinos = 1
mnu     = 0.0006  #eV
Neff    = 2.046 # -1 if massive neutrino present
standard_neutrino_neff=Neff+num_massive_neutrinos
h      = 0.67
A_s     = 2.1064e-09
n_s     = 0.96
sigmaz  = 0.03 #0.03  # redshift error
sigma8 = 0.816

b = 1.6

# list of parameters with respect to which we take derivatives
fieldnames = ['h', '\\Omega_m', '\\Omega_b', 'N_{eff}', 'm_{\\nu}', 'T_{CMB}', 'Y_{He}', '\\sigma_8', 'n_s', 'w_0', 'w_a', 'c_s^2', 'b1', '\\sigma_z']
fiducial_cosmo_param = jnp.array([h, Omegam, Omegab, Neff, mnu, Tcmb, YHe, sigma8, n_s,  w_DE_0, w_DE_a, cs2_DE, b, sigmaz])

# selected_vars = [0,1,2,7,8,9,10]
selected_vars = [1,2,9,10,0,8,7]

# modes to sample
nmodes = 256
nmu    = 128
kmin   = 2e-5 # in h/Mpc
kmax   = 2.0  # in h/Mpc


In [4]:
def Pkmu_of_cosmo( args ):
    """ Compute the matter (b+c) power spectrum for a given set of cosmological parameters"""
    param = {}
    h = args[0]
    param['Omegam'] = args[1]
    param['Omegab'] = args[2]
    param['OmegaDE'] = 1-args[1]
    param['Omegak'] = 0.0
    param['A_s'] = 2.1064e-09
    param['n_s'] = args[8]
    param['H0'] = args[0]*100
    param['Tcmb'] = args[5]
    param['YHe'] = args[6]
    param['Neff'] = args[3] 
    param['Nmnu'] = num_massive_neutrinos
    param['mnu'] = args[4]
    param['w_DE_0'] = args[9]
    param['w_DE_a'] = args[10]
    param['cs2_DE'] = args[11]
    
    sigma8 = args[7]

    param['k_p']  = 0.05
    sigmaz = args[13]
    
    ## Compute Background evolution
    param = evolve_background(param=param, thermo_module='RECFAST')

    # Compute Perturbations
    # lmaxg  = 12
    # lmaxgp = 12
    # lmaxr  = 17
    # lmaxnu = 17
    # nqmax  = 15

    rtol   = 1e-4
    atol   = 1e-4

    aexp_out = jnp.array([aexp, 1.0])
    
    print(aexp, b)
    # kmodes   = jnp.geomspace(kmin*h, kmax*h, nmodes) # kmodes is in 1/Mpc, kmin/max is in h/Mpc

    y, kmodes = evolve_perturbations( param=param, kmin=kmin*h, kmax=kmax*h, num_k=nmodes, aexp_out=aexp_out,
                                    #   lmaxg=lmaxg, lmaxgp=lmaxgp, lmaxr=lmaxr, lmaxnu=lmaxnu, nqmax=nqmax,
                                       rtol=rtol, atol=atol )

    Pm0 = get_power( k=kmodes, y=y[:,-1,:], param=param, idx=4 )
    Wth = lambda kR : 3*(jnp.sin(kR)-kR*jnp.cos(kR))/(kR)**3
    dsigma = Wth(kmodes*8.0/h)**2 * Pm0 * kmodes**3
    # sigma8_2_measured = jnp.trapz( y=(dsigma), x=jnp.log(kmodes) ) / (2*jnp.pi**2)
    sigma8_2_measured = jax.scipy.integrate.trapezoid( y=(dsigma), x=jnp.log(kmodes) ) / (2*jnp.pi**2) # for newer versions of jax

    Pkmu, mu, theta = power_Kaiser( y=y[:,0,:], kmodes=kmodes, b=b, sigma_z0 = sigmaz, aexp=aexp_out[0], nmu=nmu, param=param )
    Pkmu *= sigma8**2/ sigma8_2_measured 

    # print( 'sigma8 = ', sigma8, ', sigma8_measured = ', np.sqrt(sigma8_2_measured))
    
    return jnp.log(Pkmu*h**3), Pkmu*h**3, mu, theta, kmodes/h


In [5]:
aexp = 1. / (1. + 1.0)
b = 1.46
logPkmu0, Pkmu0, mu0, theta0, kmodes0 = Pkmu_of_cosmo( fiducial_cosmo_param )

aexp = 1. / (1. + 1.2)
b = 1.61
logPkmu1, Pkmu1, mu1, theta1, kmodes1 = Pkmu_of_cosmo( fiducial_cosmo_param )

aexp = 1. / (1. + 1.4)
b = 1.75
logPkmu2, Pkmu2, mu2, theta2, kmodes2 = Pkmu_of_cosmo( fiducial_cosmo_param )

aexp = 1. / (1. + 1.65)
b = 1.90
logPkmu3, Pkmu3, mu3, theta3, kmodes3 = Pkmu_of_cosmo( fiducial_cosmo_param )

0.5 1.46
0.45454545454545453 1.61
0.4166666666666667 1.75
0.37735849056603776 1.9


In [6]:
aexp = 1. / (1. + 1.0)
b = 1.46
dPkmu_dcosmo = jax.jacfwd( Pkmu_of_cosmo )
dlogPkmu0,dPkmu0,_,_,_ = dPkmu_dcosmo( fiducial_cosmo_param )

aexp = 1. / (1. + 1.2)
b = 1.61
dPkmu_dcosmo = jax.jacfwd( Pkmu_of_cosmo )
dlogPkmu1,dPkmu1,_,_,_ = dPkmu_dcosmo( fiducial_cosmo_param )

aexp = 1. / (1. + 1.4)
b = 1.75
dPkmu_dcosmo = jax.jacfwd( Pkmu_of_cosmo )
dlogPkmu2,dPkmu2,_,_,_ = dPkmu_dcosmo( fiducial_cosmo_param )

aexp = 1. / (1. + 1.65)
b = 1.90
dPkmu_dcosmo = jax.jacfwd( Pkmu_of_cosmo )
dlogPkmu3,dPkmu3,_,_,_ = dPkmu_dcosmo( fiducial_cosmo_param )

0.5 1.46
0.45454545454545453 1.61
0.4166666666666667 1.75
0.37735849056603776 1.9


In [9]:
Vs_bin = [7.94e9, 9.15e9, 10.05e9, 16.22e9]  # Vs_bin has units Mpc**3 / h**3
Dz_bin = [0.2, 0.2, 0.2, 0.3]                # unitless
dNdOmega = [1815.0, 1701.5, 1410.0, 940.97]  # units are 1/deg**2 

def effective_volume(Pkmu, nbin=0 ):
    Vs = Vs_bin[nbin] 
    nz = dNdOmega[nbin] * 15000 / Vs * Dz_bin[nbin]  
    Veff = Vs * (nz * Pkmu  / (nz * Pkmu  + 1))**2
    return Veff

F = jnp.zeros((len(selected_vars),len(selected_vars)))

#### To solve integral over mu, use jnp.trapz over mu dimension of Pkmu

for idx, Pkmu, dlogPkmu, kmodes in zip([0,1,2,3],[Pkmu0,Pkmu1,Pkmu2,Pkmu3],[dlogPkmu0,dlogPkmu1,dlogPkmu2,dlogPkmu3],[kmodes0,kmodes1,kmodes2,kmodes3]):
    for i,ivar in enumerate(selected_vars): 
        for j,jvar in enumerate(selected_vars):

            Veff = effective_volume(Pkmu, nbin=idx )
            # product of derivatives
            dPkmu_dcosmo_prod = dlogPkmu[...,ivar]*dlogPkmu[...,jvar] * Veff 
            dPkmu_dcosmo_prod = dPkmu_dcosmo_prod.at[ jnp.isnan(dPkmu_dcosmo_prod) ].set( -10.0 )

            # integrate over mu
            n = len(mu1)
            # dPk_dcosmo = jnp.trapz( dPkmu_dcosmo_prod[...,:n//2], x=mu1[...,:n//2], axis=-1 )
            dPk_dcosmo = jax.scipy.integrate.trapezoid( dPkmu_dcosmo_prod[...,:n//2], x=mu1[:n//2], axis=-1 )

            # integrate over k
            # F_ij = 1/(8*jnp.pi**2) * jnp.trapz( dPk_dcosmo * kmodes**3, x=jnp.log(kmodes), axis=-1 )
            F_ij = 1/(8*jnp.pi**2) * jax.scipy.integrate.trapezoid( dPk_dcosmo * kmodes**3, x=jnp.log(kmodes), axis=-1 )

            F = F.at[i,j].add( F_ij )

In [10]:
def plot_uncertainty_ellipse(F, i, j, x0, y0, ax, color='b'):
    """
    Plot uncertainty ellipse given Fisher matrix F.
    
    Parameters:
        - F: Fisher information matrix (2x2 array).
    """
    
    # Calculate covariance matrix
    C = jnp.linalg.inv(F)  # arXiv:1910.09273, eq. (44)
    Cii, Cjj, Cij = C[i,i], C[j,j], C[i,j]

    a = jnp.sqrt(jnp.abs((Cii+Cjj)/2 + jnp.sqrt(0.25*(Cii-Cjj)**2 + Cij**2)))  # arXiv:1910.09273, eq. (47)
    b = jnp.sqrt(jnp.abs((Cii+Cjj)/2 - jnp.sqrt(0.25*(Cii-Cjj)**2 + Cij**2)))  # ...
    phi = 0.5 * jnp.arctan2( 2*Cij, Cii-Cjj )                                  # ...

    # confidence region amplitdues, from arXiv:1910.09273, below eq. (47):
    f1 = jnp.sqrt(2.3)   # 1 sigma
    f2 = jnp.sqrt(6.18)  # 2 sigma
    f3 = jnp.sqrt(11.8)  # 3 sigma

    ax.add_patch( Ellipse((x0,y0),f1*a*2,f1*b*2,angle=np.degrees(phi),alpha=0.2, facecolor=color) )
    ax.add_patch( Ellipse((x0,y0),f2*a*2,f2*b*2,angle=np.degrees(phi),alpha=0.2, facecolor=color) )
    
    ax.plot(x0,y0,'k+',lw=0.125,ms=2)

In [13]:
# load Euclid collaboration Fisher matrix data
data = np.load('data/fisher_matrix_euclid.npz')
euclid_vars =  ['Omegam', 'Omegab', 'w0', 'wa', 'h', 'ns', 'sigma8']
euclid_reorder = range(7)
Feuclid = data['F']

In [14]:
fig, ax = plt.subplots( len(euclid_vars), len(euclid_vars), figsize=(10,10), layout='constrained')
FF = jnp.linalg.inv(Feuclid)
for i, label1 in enumerate(euclid_vars):
  ivar = selected_vars[i]
  for j, label2 in enumerate(euclid_vars):
    jvar = selected_vars[j]
    plot_uncertainty_ellipse( FF, euclid_reorder[i], euclid_reorder[j], fiducial_cosmo_param[ivar], fiducial_cosmo_param[jvar], ax[i,j], color=[0,0,1] )
    plot_uncertainty_ellipse( F, i, j, fiducial_cosmo_param[ivar], fiducial_cosmo_param[jvar], ax[i,j], color=[0,0.5,0] )



    ax[i,j].set_xlabel(f'${fieldnames[ivar]}$', size=8)
    ax[i,j].set_ylabel(f'${fieldnames[jvar]}$', size=8)
    
    ax[i,j].tick_params(axis='both', which='both', labelsize=5)